In [5]:
#download and install CBC solver
#%pip install pyomo-windows

# from pyomo_windows.solvers import DownloadSolvers
# downloader = DownloadSolvers()
# downloader.download_cbc()

In [6]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler
import importlib
from collections import defaultdict
from pyomo.environ import *
%matplotlib inline
import ParameterDataLoader as ParameterDataLoader_Module
import MultiCriteriaMIPModel as MultiCriteriaMIPModel_Module

importlib.reload(ParameterDataLoader_Module) # in case of updates
importlib.reload(MultiCriteriaMIPModel_Module) # in case of updates

from ParameterDataLoader import ParameterDataLoader
from MultiCriteriaMIPModel import MultiCriteriaMIPModel

#when ALPHA is almost equal to BETA, the solver struggles to find an optimal feasible solution & it converges slowly
H_FIXED_MINUTES = 480
ALPHA = 1.0 #makespan weight
BETA = 1.0 #operator activation weight 
BIG_M = 1e5

### Load Mission Batch Datasets
For now, we introduce some simplifications:
- we don't consider the priority of mission, neither the area of mission.
- we don't consider the operational area of forklift.
- we don't consider the tare of mission's pallet (UDC). 
- we suppose that the Z-axis feature of mission represents the exact meters that forklift sould reach vertically; otherwise, we might need a scaling coefficient which reflects the warehouse's real-world conversion of Z-axis units in meter. For such assumption, the features "FROM_Z" and "TO_Z" need to be standarized.
- the travel time is calculated using forklift's overall average speed, not for each single forklift.

*Mission types are {loading, unloading, internal repositioning}.

#### Mission Travel Time Calculation
The travel time between each pair of missions (ordered pair) is pre-calculated using the heuristic algorithm A* w.r.t. the loaded warehouse's map.

#### Mission Processing Time Calculation
A warehouse is organized in various areas, where each area has a set of levels over Z-axis on different sides, and each level has a set of cells. So in order reach a specific cell, we need to know its coordinated $(x, y, z)$. the time required to reach the source point $(x ,y)$ of a mission is already calculated in the previous point. To which we need to calculate the time relative to Z-axis.

The processing time of a mission (pallet) is calculated w.r.t. the characteristics of each operator considering these elements:
* the time required to lift the fork up over the Z-axis in order to load the pallet.
* the time required to place the pallet on the fork. [ignored for simplification]
* the time required to move the pallet from its source point $(x, y)$ to a destination point $(x', y')$. (for which we're already given the distance from source point to destination point. the distance is calculated like other pre-calculated distances in "ref. Mission Travel Time Calculation").
* the time required to lift the fork up over the Z-axis in order to unload the pallet.
* the time required to place the pallet off the fork. [ignored for simplification]

#### Operator Skill Score Calculation
The skill score is a measure of how the fork lift is adequate to pallet type.
If the pallet dimensions excced forks dimension w.r.t. the threshold, negative Big_M is assigned as operator skill for such pallet type.
lower the difference between pallet dimensions and fork dimensions, higher the skill score.
Big_M can be used to normalize the skill score between 0 and 1.

In [7]:
MISSION_BATCH_DIR = "./datasets/Batch10M_distanced.csv"
UDC_TYPES_DIR = "./datasets/WM_UDC_TYPE.csv"
MISSION_BATCH_TRAVEL_DIR = "./datasets/Batch100M_travel_distanced.csv"
FORK_LIFTS_DIR = "./datasets/ForkLifts10W.csv"
#MISSION_TYPES_DIR = "./datasets/MissionTypes.csv"
SCHEDULE_DIR = "./schedules/"
BATCH_NAME = MISSION_BATCH_DIR.replace('./datasets/Batch', '').replace('_distanced.csv', '')

#with mission TP_UDC, we can retreive relative width and length from mission types
#mission_batch_features = ['CD_MISSION', 'TP_MISSION', 'FROM_X', 'FROM_Y', 'TO_X', 'TO_Y', 'TP_UDC', 'DISTANCE']
mission_batch_features = ['CD_MISSION', 'FROM_X', 'FROM_Y', 'TO_X', 'TO_Y', 'FROM_Z', 'TO_Z', 'TP_UDC', 'DISTANCE']
udc_types_features = ['TP_UDC', 'WIDTH', 'LENGTH']
mission_batch_travel_features = ['CD_MISSION_1', 'CD_MISSION_2', 'FROM_X', 'FROM_Y', 'TO_X', 'TO_Y', 'DISTANCE']
fork_lifts_features = ['OID', 'FORK_WIDTH', 'FORK_LENGTH', 'SPEED', 'SPEED_WITH_LOAD', 'UP_SPEED', 'UP_SPEED_WITH_LOAD', 'DOWN_SPEED', 'DOWN_SPEED_WITH_LOAD']
#fork_lifts_features = ['OID', 'FORK_WIDTH', 'FORK_LENGTH', 'SPEED', 'SPEED_WITH_LOAD']
#mission_types_features = ['TP_MISSION', 'DSC_MISSION']

mission_batch_df = pd.read_csv(MISSION_BATCH_DIR)[mission_batch_features]
udc_types_df = pd.read_csv(UDC_TYPES_DIR)[udc_types_features]
mission_batch_travel_df = pd.read_csv(MISSION_BATCH_TRAVEL_DIR)[mission_batch_travel_features]
fork_lifts_df = pd.read_csv(FORK_LIFTS_DIR)[fork_lifts_features]
#mission_types_df = pd.read_csv(MISSION_TYPES_DIR)[mission_types_features]

mission_batch_df.head()

,CD_MISSION,FROM_X,FROM_Y,TO_X,TO_Y,FROM_Z,TO_Z,TP_UDC,DISTANCE
0,3911722,246,426,194,373,0,0,1,59
1,3911727,246,426,194,373,0,0,1,59
2,3911733,246,426,194,373,0,0,1,59
3,3911740,262,329,194,373,0,0,1,103
4,3911742,246,426,194,373,0,0,1,59


#### Preprocessing of missions
We have chosen standard scaling to standarized the features relative to Z-axis. Standard Scaling transforms the data such that it has a mean ($\mu$) of 0 and a standard deviation ($\sigma$) of 1. The formula for the transformation is:
$$z = \frac{x − \mu}{\sigma}​$$

In [8]:
#scale only FROM_Z and TO_Z columns
features_to_scale = ['FROM_Z','TO_Z']
df_to_scale = mission_batch_df[features_to_scale]

scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_to_scale)

df_scaled_features = pd.DataFrame(
    scaled_data,
    columns=features_to_scale,
    index=mission_batch_df.index
)

df_scaled_features = df_scaled_features.clip(lower=0)
#df_scaled_features.head()

df_unscaled_features = mission_batch_df.drop(columns=features_to_scale)

mission_batch_df_scaled = pd.concat([df_unscaled_features, df_scaled_features], axis=1)

mission_batch_df_scaled.head()

,CD_MISSION,FROM_X,FROM_Y,TO_X,TO_Y,TP_UDC,DISTANCE,FROM_Z,TO_Z
0,3911722,246,426,194,373,1,59,0.0,0.0
1,3911727,246,426,194,373,1,59,0.0,0.0
2,3911733,246,426,194,373,1,59,0.0,0.0
3,3911740,262,329,194,373,1,103,0.0,0.0
4,3911742,246,426,194,373,1,59,0.0,0.0


##### Mission feature units
- distance, width and length in meter.
Note that the distance is already pre-calculated using external program by applying A* on features {FORM_X, FROM_Y, TO_X, TO_Y}. The external progam has considered an image that describe the real-world warehouse map, then the path is estimated by A* after scaling image's pixels to calculate the distance approximately.  

*A future efficient approach could be through saving the warehouse map on a geografic system exploiting GIS queries for distance and path calculations.

In [9]:
BASE_MISSION = [0, 0, 0, 0, 0, 0, 0, 0, 0]  #virtual base mission for operators to start and end their routes

#in case of merging WIDTH and LENGTH from UDC types into mission batch, so there'd be no need to access UDC types during optimization
# mission_batch_df_scaled = pd.merge(mission_batch_df_scaled, udc_types_df, on='TP_UDC')
# mission_batch_df_scaled.drop(columns=['TP_UDC'], inplace=True)

mission_batch_df_with_base = pd.concat([pd.DataFrame([BASE_MISSION], columns=mission_batch_df_scaled.columns), mission_batch_df_scaled], ignore_index=True)
mission_batch_df_scaled['TP_UDC'].fillna(udc_types_df.iloc[0]['TP_UDC'], inplace=True) #the udc_type base mission will remain 0
mission_batch_df_with_base.head()

C:\Users\abudo\AppData\Local\Temp\ipykernel_68928\3641040282.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  mission_batch_df_scaled['TP_UDC'].fillna(udc_types_df.iloc[0]['TP_UDC'], inplace=True) #the udc_type base mission will remain 0


,CD_MISSION,FROM_X,FROM_Y,TO_X,TO_Y,TP_UDC,DISTANCE,FROM_Z,TO_Z
0,0,0,0,0,0,0,0,0.0,0.0
1,3911722,246,426,194,373,1,59,0.0,0.0
2,3911727,246,426,194,373,1,59,0.0,0.0
3,3911733,246,426,194,373,1,59,0.0,0.0
4,3911740,262,329,194,373,1,103,0.0,0.0


In [10]:
#remove pallet types not present in the mission batch
udc_types_in_mission_batch = mission_batch_df_scaled['TP_UDC'].unique().tolist()
udc_types_df = udc_types_df[udc_types_df['TP_UDC'].isin(udc_types_in_mission_batch)]
udc_types_df.head()

,TP_UDC,WIDTH,LENGTH
7,1,0.8,1.2


In [11]:
mission_batch_travel_df.head()

,CD_MISSION_1,CD_MISSION_2,FROM_X,FROM_Y,TO_X,TO_Y,DISTANCE
0,3911722,3911727,194,373,246,426,59
1,3911727,3911722,194,373,246,426,59
2,3911722,3911733,194,373,246,426,59
3,3911733,3911722,194,373,246,426,59
4,3911722,3911740,194,373,262,329,99


##### Fork Lift feature units
- width & length in meter.
- speed in meter/minute.

In [12]:
fork_lifts_df.head()

,OID,FORK_WIDTH,FORK_LENGTH,SPEED,SPEED_WITH_LOAD,UP_SPEED,UP_SPEED_WITH_LOAD,DOWN_SPEED,DOWN_SPEED_WITH_LOAD
0,1,1.1,1.7,300.0,300.0,34.8,31.2,24.0,28.8
1,2,1.1,1.7,300.0,300.0,34.8,31.2,24.0,28.8
2,3,1.1,1.7,300.0,300.0,34.8,31.2,24.0,28.8
3,4,1.1,1.7,300.0,300.0,34.8,31.2,24.0,28.8
4,5,1.1,1.7,300.0,300.0,34.8,31.2,24.0,28.8


In [13]:
#simple problem from the dat file to load the concreteModel
USE_SIMPLE_PROBLEM = False

if USE_SIMPLE_PROBLEM:
    missions = [1, 2, 3, 4]
    operators = ['A', 'B']
    pallet_types = [1, 2]
    missions_with_base = [0, 1, 2, 3, 4]

    processing_times = {
        ('A', 1): 20, ('A', 2): 15, ('A', 3): 30, ('A', 4): 18,
        ('B', 1): 25, ('B', 2): 10, ('B', 3): 20, ('B', 4): 22
    }

    travel_times = {
        (0, 1): 5, (0, 2): 8, (0, 3): 4, (0, 4): 7,
        (1, 0): 6, (1, 2): 3, (1, 3): 7, (1, 4): 2,
        (2, 0): 4, (2, 1): 3, (2, 3): 5, (2, 4): 4,
        (3, 0): 5, (3, 1): 7, (3, 2): 5, (3, 4): 9,
        (4, 0): 7, (4, 1): 2, (4, 2): 4, (4, 3): 9
    }

    skill_scores = {
        ('A', 1): 20, ('A', 2): 15,
        ('B', 1): 25, ('B', 2): 10
    }

    mission_pallet_types = {
        (1, 1): 1, (1, 2): 0,
        (2, 1): 0, (2, 2): 1,
        (3, 1): 1, (3, 2): 0,
        (4, 1): 0, (4, 2): 1
    }

    print(f'missions: {missions}')
    print(f'missions_with_base: {missions_with_base}')
    print(f'pallet_types: {pallet_types}')
    print(f'operators: {operators}')
    print(f'travel_times: {travel_times}')
    print(f'processing_times: {processing_times}')
    print(f'skill_scores: {skill_scores}')
    print(f'mission_pallet_types: {mission_pallet_types}')

In [14]:
if not USE_SIMPLE_PROBLEM:
    parameter_data_loader = ParameterDataLoader(
        mission_batch_df_scaled,
        mission_batch_df_with_base,
        mission_batch_travel_df,
        fork_lifts_df,
        udc_types_df,
        BIG_M
    )

    missions = mission_batch_df_scaled['CD_MISSION'].astype(int).to_list()
    operators = fork_lifts_df['OID'].astype(int).to_list()
    pallet_types = udc_types_df['TP_UDC'].astype(int).to_list()
    missions_with_base = mission_batch_df_with_base['CD_MISSION'].to_list()

    travel_times = parameter_data_loader.get_mission_travel_times()
    processing_times = parameter_data_loader.get_mission_processing_times()
    skill_scores = parameter_data_loader.get_operator_skill_scores()
    mission_pallet_types = parameter_data_loader.get_mission_pallet_types()

    print(f'missions: {missions}')
    print(f'missions_with_base: {missions_with_base}')
    print(f'pallet_types: {pallet_types}')
    print(f'operators: {operators}')
    print(f'travel_times: {travel_times}')
    print(f'processing_times: {processing_times}')
    print(f'skill_scores: {skill_scores}')
    print(f'mission_pallet_types: {mission_pallet_types}')


missions: [3911722, 3911727, 3911733, 3911740, 3911742, 3911746, 3911750, 3911753, 3911755, 3912155]
missions_with_base: [0, 3911722, 3911727, 3911733, 3911740, 3911742, 3911746, 3911750, 3911753, 3911755, 3912155]
pallet_types: [1]
operators: [1, 2, 3, 4, 5, 37, 38, 39, 40, 41]
travel_times: {(3911722, 3911727): np.float64(5.164650396974897), (3911727, 3911722): np.float64(5.164650396974897), (3911722, 3911733): np.float64(5.164650396974897), (3911733, 3911722): np.float64(5.164650396974897), (3911722, 3911740): np.float64(5.164650396974897), (3911740, 3911722): np.float64(5.164650396974897), (3911722, 3911742): np.float64(5.164650396974897), (3911742, 3911722): np.float64(5.164650396974897), (3911722, 3911746): np.float64(5.164650396974897), (3911746, 3911722): np.float64(5.164650396974897), (3911722, 3911750): np.float64(5.750694182817754), (3911750, 3911722): np.float64(5.750694182817754), (3911722, 3911753): np.float64(5.750694182817754), (3911753, 3911722): np.float64(5.750694182

c:\Users\abudo\source\vscode_projects\ManualWarehouseOptimization\ParameterDataLoader.py:27: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  self.fork_lifts_df['SPEED'].fillna(300, inplace=True)
c:\Users\abudo\source\vscode_projects\ManualWarehouseOptimization\ParameterDataLoader.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.fork_lif

In [15]:
#necessary assertations to ensure data consistency before optimization

mission_travel_times = defaultdict(list)
mission_processing_times = defaultdict(list)
pallet_type_skill_scores = defaultdict(list)

#(cd_mission, cd_mission): travel_time
{mission_travel_times[k[0]].append(travel_time) for k, travel_time in travel_times.items()}
total_travel_mins = sum([min(p_time) for mission, p_time in mission_travel_times.items()])
total_travel_maxs = sum([max(p_time) for mission, p_time in mission_travel_times.items()])

#(oid_fork_lift, cd_mission): processing_time
{mission_processing_times[k[1]].append(processing_time) for k, processing_time in processing_times.items()}
total_processing_mins = sum([min(p_time) for mission, p_time in mission_processing_times.items()])
total_processing_maxs = sum([max(p_time) for mission, p_time in mission_processing_times.items()])

#(oid_fork_lift, pallet_type): skill_score
{pallet_type_skill_scores[k[1]].append(skill_score) for k, skill_score in skill_scores.items() if skill_score > 0}

assert (total_travel_mins + total_processing_mins) <= H_FIXED_MINUTES * len(fork_lifts_df),\
"Total estimated minimum required time for all missions exceeds total available operator time."

pallet_types = udc_types_df["TP_UDC"].to_list()
assert len([1 for mission_pallet_type in mission_batch_df_scaled["TP_UDC"].tolist() if mission_pallet_type not in pallet_types]) == 0,\
"Each mission's pallet type needs to be in the list of pallet types."

assert all([len(skill_scores_list) > 0 for skill_scores_list in pallet_type_skill_scores.values()]),\
"Each pallet type needs to be in at least one operator score."


In [16]:
#calculate M_value dynamically for Big-M_Time used in constraints

# P_series = pd.Series(processing_times)
# max_P_per_mission = P_series.groupby(level=1).max()
# sum_max_P = max_P_per_mission.sum()

# T_series = pd.Series(travel_times)
# sum_T = T_series.sum()

# M_value = sum_max_P + sum_T + 1.0
#OR use maximum possible time based on maximum travel and processing times
M_value = total_travel_maxs + total_processing_maxs + 1.0
print("Calculated M_value for Big-M_Time is: ", M_value)



Calculated M_value for Big-M_Time is:  122.81234543095148


In [17]:
mcmModel = MultiCriteriaMIPModel(missions,
                                 operators,
                                 pallet_types,
                                 missions_with_base,
                                 processing_times,
                                 travel_times,
                                 skill_scores,
                                 mission_pallet_types,
                                 H_FIXED_MINUTES,
                                 ALPHA,
                                 BETA,
                                 M_value,
                                 BIG_M
                                )

#mcmModel= MultiCriteriaMIPModel()

In [18]:
#instance, results = mcmModel.solve("Mip_parameters.dat", "cbc")
#instance, results = mcmModel.solve(data_file="Mip_parameters.dat", solver_name="cplex")
#instance, results = mcmModel.solve()
instance, results = mcmModel.solve(solver_name="cplex_direct")
#instance, results = mcmModel.solve(solver_name="cplex_persistent")

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Tried aggregator 3 times.
MIP Presolve eliminated 2810 rows and 130 columns.
MIP Presolve modified 100 coefficients.
Aggregator did 1800 substitutions.
Reduced MIP has 2359 rows, 1331 columns, and 9528 nonzeros.
Reduced MIP has 1210 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (18.91 ticks)
Found incumbent of value 485.000000 after 0.02 sec. (26.91 ticks)
Probing fixed 1 vars, tightened 0 bounds.
Probing time = 0.02 sec. (18.81 ticks)
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve eliminated 11 rows and 1 columns.
Reduced MIP has 2348 rows, 1330 columns, and 9504 nonzeros.
Reduced MIP has 1209 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (8.10 ticks)
Probing time = 0.00 sec. (5.11 ticks)
Clique table members: 5555.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mod

In [19]:
mcmModel.display_solution(instance)

Model unknown

  Variables:
    y : Size=10, Index=I_max
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   1.0 :     1 : False : False : Binary
          2 :     0 :   1.0 :     1 : False : False : Binary
          3 :     0 :   1.0 :     1 : False : False : Binary
          4 :     0 :   1.0 :     1 : False : False : Binary
          5 :     0 :   1.0 :     1 : False : False : Binary
         37 :     0 :   1.0 :     1 : False : False : Binary
         38 :     0 :   1.0 :     1 : False : False : Binary
         39 :     0 :   1.0 :     1 : False : False : Binary
         40 :     0 :   1.0 :     1 : False : False : Binary
         41 :     0 :   1.0 :     1 : False : False : Binary
    z : Size=1210, Index=I_max*J_prime*J_prime
        Key                    : Lower : Value : Upper : Fixed : Stale : Domain
                     (1, 0, 0) :     0 :  None :     1 : False :  True : Binary
               (1, 0, 3911722) :     0 :  -0.0 :     1 : False : 

In [20]:
instance = mcmModel.model #in case of concrete model already solved

solution_data = []
for i in instance.I_max:
    if value(instance.y[i]) == 1: #in case of unfeasible solution, it launches a pyomo error
        #operator i is active
        for j in instance.J_prime:
            for k in instance.J_prime:
                
                #apply the j != k filter to avoid pyomo warning about self-loops
                if j != k:
                    #check if the index (i, j, k) is valid and exists in z
                    if (i, j, k) in instance.z:
                        
                        #check for active flow
                        if value(instance.z[i, j, k]) > 0:
                            
                            #add data only for flow between service orders (not flow to/from Base)
                            if j in instance.J:
                                solution_data.append({
                                    'Operator': i,
                                    'Task': j,
                                    'Start': value(instance.S[j]),
                                    'Finish': value(instance.C[j]),
                                    'Successor': k
                                })

df_schedule = pd.DataFrame(solution_data)
if not df_schedule.empty:
    df_schedule = df_schedule.sort_values(by=['Operator', 'Start']).reset_index(drop=True)
    try:
        #save the schedule to a CSV file
        filename = f'{SCHEDULE_DIR}schedule{BATCH_NAME}.csv'
        df_schedule.to_csv(filename, index=False)
        print(f"\nSuccessfully saved schedule to {filename}")
    except Exception as e:
        print(f"Error saving to CSV: {e}")

df_schedule


Successfully saved schedule to ./schedules/schedule10M.csv


,Operator,Task,Start,Finish,Successor
0,1,3911755,0.000000e+00,5.660000,0
1,2,3911740,4.534603e-01,5.796794,0
2,3,3911750,0.000000e+00,5.660000,0
3,4,3911722,0.000000e+00,5.196667,0
4,5,3911753,0.000000e+00,5.660000,0
5,37,3911742,6.551948e-01,5.796794,0
6,38,3912155,-1.172396e-13,5.796794,0
7,39,3911727,0.000000e+00,5.141599,0
8,40,3911733,6.551948e-01,5.796794,0
9,41,3911746,0.000000e+00,5.141599,0


In [21]:
routes = {} 
    
# Check for solution status (optional but recommended)
# if not (results.solver.status == SolverStatus.ok and results.solver.termination_condition == TerminationCondition.optimal):
#     print("Solver did not find an optimal solution.")
#     return

# Call the function with your solved instance
# print_optimal_routes(instance)

# 1. Iterate over all potential operators
for i in instance.I_max:
    if value(instance.y[i]) < 0.5: #in case of unfeasible solution, it launches a pyomo error
        continue #skip unactivated operators

    print(f"\n--- Operator {i} (Activated) ---")

    current_node = 0  # Start at the Base node
    route_sequence = []
    is_route_complete = False

    #security counter to prevent infinite loops (should not happen if flow constraints are correct)
    max_steps = len(instance.J) + 2 
    steps = 0

    #loop until the route returns to the Base (k=0)
    while not is_route_complete and steps < max_steps:
        steps += 1
        
        #search for the next step (k) starting from the current node (current_node)
        found_next_step = False
        for k in instance.J_prime:
            if current_node == k:
                continue # Skip self-loop
            
            try:
                #check if the arc (current_node -> k) is active
                if value(instance.z[i, current_node, k]) > 0.5:
                    
                    #calculate travel time for printing
                    travel_time = value(instance.T[current_node, k])
                    
                    #handle Movement Types
                    if k == 0:
                        #final movement: Return to Base
                        route_sequence.append(f"-> Base (Arc: {current_node} -> 0 | Travel: {travel_time:.2f})")
                        is_route_complete = True
                        break #exit the k loop
                    else:
                        #service movement: j -> k
                        start_time = value(instance.S[k])
                        finish_time = value(instance.C[k])
                        proc_time = finish_time - start_time
                        
                        movement_detail = (
                            f"-> Order {k} | Travel: {travel_time:.2f} | Start: {start_time:.2f} | "
                            f"Proc: {proc_time:.2f} | Finish: {finish_time:.2f}"
                        )
                        route_sequence.append(movement_detail)
                        
                        #move to the next node in the sequence
                        current_node = k
                        found_next_step = True
                        break #exit the k loop
                        
            except KeyError:
                #this node pair might not exist in the defined set of z variables (e.g., if filtered by a complex index)
                continue

        if not found_next_step and not is_route_complete:
            print(f"Error: Route stopped unexpectedly at node {current_node} for operator {i}.")
            break
        
    #print the final sequenced route for the operator
    route_string = "\n".join(route_sequence)
    print("Path: Base " + route_string)
    #print(f"Total Time (C_last): {value(instance.C_last[i]):.2f}")
    print("-" * 40)


--- Operator 1 (Activated) ---
Path: Base -> Order 3911755 | Travel: 6.20 | Start: 0.00 | Proc: 5.66 | Finish: 5.66
-> Base (Arc: 3911755 -> 0 | Travel: 6.20)
----------------------------------------

--- Operator 2 (Activated) ---
Path: Base -> Order 3911740 | Travel: 5.97 | Start: 0.45 | Proc: 5.34 | Finish: 5.80
-> Base (Arc: 3911740 -> 0 | Travel: 5.97)
----------------------------------------

--- Operator 3 (Activated) ---
Path: Base -> Order 3911750 | Travel: 6.20 | Start: 0.00 | Proc: 5.66 | Finish: 5.66
-> Base (Arc: 3911750 -> 0 | Travel: 6.20)
----------------------------------------

--- Operator 4 (Activated) ---
Path: Base -> Order 3911722 | Travel: 6.20 | Start: 0.00 | Proc: 5.20 | Finish: 5.20
-> Base (Arc: 3911722 -> 0 | Travel: 6.20)
----------------------------------------

--- Operator 5 (Activated) ---
Path: Base -> Order 3911753 | Travel: 6.20 | Start: 0.00 | Proc: 5.66 | Finish: 5.66
-> Base (Arc: 3911753 -> 0 | Travel: 6.20)
------------------------------------

In [22]:
#initialize list to store all route data
solution_data = []

#iterate over all potential operators
for i in instance.I_max:
    #skip unactivated operators (using tolerance check)
    if value(instance.y[i]) < 0.5:
        continue 

    #start at the Base node (0)
    current_node = 0  
    is_route_complete = False
    
    max_steps = len(instance.J) + 2 
    steps = 0
    sequence_rank = 1  #to keep track of the order of visits

    #loop until the route returns to the Base (k=0)
    while not is_route_complete and steps < max_steps:
        steps += 1
        found_next_step = False
        
        #search for the next step (k) starting from current_node
        for k in instance.J_prime:
            if current_node == k:
                continue # Skip self-loops
            
            #check if arc (current_node -> k) is active in variable z
            #we use try/except or direct check if (i, current_node, k) in instance.z
            if (i, current_node, k) in instance.z and value(instance.z[i, current_node, k]) > 0.5:
                
                travel_time = value(instance.T[current_node, k])
                
                #common row data
                row = {
                    'Operator_ID': i,
                    'Sequence_Rank': sequence_rank,
                    'From_Node': current_node,
                    'To_Node': k,
                    'Travel_Time': travel_time,
                    'Start_Service': None,
                    'Processing_Time': None,
                    'Finish_Service': None,
                    'Activity_Type': 'Travel'
                }

                if k == 0:
                    #final movement: Return to Base
                    row['Activity_Type'] = 'Return to Base'
                    
                    solution_data.append(row)
                    is_route_complete = True
                    found_next_step = True
                    break #exit the k loop
                
                else:
                    #service movement: Visit Task k
                    start_time = value(instance.S[k])
                    finish_time = value(instance.C[k])
                    proc_time = finish_time - start_time
                    
                    #update row with service details
                    row['Activity_Type'] = 'Service'
                    row['Start_Service'] = start_time
                    row['Processing_Time'] = proc_time
                    row['Finish_Service'] = finish_time
                    
                    solution_data.append(row)
                    
                    #move to next node
                    current_node = k
                    sequence_rank += 1
                    found_next_step = True
                    break #exit the k loop

        if not found_next_step and not is_route_complete:
            print(f"Error: Route stopped unexpectedly at node {current_node} for operator {i}.")
            break

df_routes = pd.DataFrame(solution_data)

if not df_routes.empty:
    #reorder columns
    cols = ['Operator_ID', 'Sequence_Rank', 'From_Node', 'To_Node', 'Activity_Type', 
            'Travel_Time', 'Start_Service', 'Finish_Service', 'Processing_Time']
    df_routes = df_routes[cols]
    
    #sort by Operator and Sequence
    df_routes = df_routes.sort_values(by=['Operator_ID', 'Sequence_Rank'])
    
    print("Preview of Export Data:")
    print(df_routes.head())

    filename = f'{SCHEDULE_DIR}schedule{BATCH_NAME}_travel.csv'
    df_routes.to_csv(filename, index=False)
    print(f"\nSuccessfully saved routes to {filename}")

else:
    print("No active routes found to export.")


Preview of Export Data:
   Operator_ID  Sequence_Rank  From_Node  To_Node   Activity_Type  \
0            1              1          0  3911755         Service   
1            1              2    3911755        0  Return to Base   
2            2              1          0  3911740         Service   
3            2              2    3911740        0  Return to Base   
4            3              1          0  3911750         Service   

   Travel_Time  Start_Service  Finish_Service  Processing_Time  
0     6.199994        0.00000        5.660000         5.660000  
1     6.199994            NaN             NaN              NaN  
2     5.965577        0.45346        5.796794         5.343333  
3     5.965577            NaN             NaN              NaN  
4     6.199994        0.00000        5.660000         5.660000  

Successfully saved routes to ./schedules/schedule10M_travel.csv
